In [1]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import numpy as np
import nibabel as nib
from scipy.ndimage import binary_dilation, generate_binary_structure
from sklearn.decomposition import PCA
from skimage.measure import marching_cubes

# Load the NIfTI file
nii_file = nib.load(r"C:\Users\邱宇辰\OneDrive - Universiteit Utrecht\桌面\Utrecht studying\Team challenge\Aneurysm_TC_data\C0001\SegmentationVA.nii")
seg_data = nii_file.get_fdata()

# Get the coordinates of voxels with values 1 and 2
x1, y1, z1 = np.where(seg_data == 1)  # Cerebral vessels
x2, y2, z2 = np.where(seg_data == 2)  # Aneurysms

# Downsampling function
def downsample_coordinates(x, y, z, factor=2):
    indices = np.arange(0, len(x), factor)
    return x[indices], y[indices], z[indices]

# Downsample coordinates
x1, y1, z1 = downsample_coordinates(x1, y1, z1, factor=2)
x2, y2, z2 = downsample_coordinates(x2, y2, z2, factor=2)



d:\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
d:\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
d:\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


The connection area between cerebral blood vessels and aneurysms is obtained by dilation operation (binary_dilation), and this area is further dilated to make the connection surface thicker. It is used to highlight the surface between cerebral blood vessels and aneurysms.

In [2]:
# Perform additional dilation on the connection area to make the connection surface thicker
def find_thick_connection_region(seg_data, iterations=2):
    # Create masks for vessels and aneurysms
    vessels_mask = (seg_data == 1)
    aneurysms_mask = (seg_data == 2)
    
    # First dilate to obtain preliminary connection area
    structure = generate_binary_structure(3, 1)  # 3D connectivity structure
    dilated_vessels = binary_dilation(vessels_mask, structure=structure)
    dilated_aneurysms = binary_dilation(aneurysms_mask, structure=structure)
    connection_region = np.logical_and(dilated_vessels, dilated_aneurysms)
    
    # Further dilate to make the connection surface thicker
    thick_connection_region = binary_dilation(connection_region, structure=structure, iterations=iterations)
    x, y, z = np.where(thick_connection_region)
    return x, y, z

# Use additional dilation for the connected area
x, y, z = find_thick_connection_region(seg_data, iterations=2)

The normal vector of the connected region is calculated using principal component analysis (PCA). PCA is used to fit the plane of the connected region, and the normal vector is the eigenvector corresponding to the smallest eigenvalue in the PCA method.

In [3]:
# Calculate the normal vector of the connected region
def calculate_normal_vector(x, y, z):
    # Combine the coordinates into a set of points
    points = np.vstack((x, y, z)).T
    
    # Use PCA to fit a plane
    pca = PCA(n_components=3)
    pca.fit(points)
    
    # The normal vector is the eigenvector corresponding to the smallest eigenvalue of PCA
    normal_vector = pca.components_[2]
    return normal_vector

# Calculate the normal vector and center point
if len(x) > 0:
    normal_vector = calculate_normal_vector(x, y, z)
    center_x, center_y, center_z = np.mean(x), np.mean(y), np.mean(z)
else:
    normal_vector = np.array([0, 0, 1])  # Default normal vector
    center_x, center_y, center_z = 0, 0, 0  # Default center point

Generating 3D mesh (i.e., model surface) for the connected region, cerebral vascular and aneurysm regions by the Marching Cubes algorithm. 

In [4]:
# Generate mesh for cerebral vessels and aneurysms
def generate_mesh(seg_data, label):
    mask = (seg_data == label)
    if np.any(mask):
        verts, faces, _, _ = marching_cubes(mask, level=0.5)
        return verts, faces
    return None, None

# Generate meshes for vessels (label 1) and aneurysms (label 2)
verts1, faces1 = generate_mesh(seg_data, 1)  # Cerebral vessels
verts2, faces2 = generate_mesh(seg_data, 2)  # Aneurysms

# Generate mesh for the (thickened) connected region
def generate_connection_mesh(x, y, z):
    if len(x) > 0:
        # Create a binary mask for the connected region
        connection_mask = np.zeros_like(seg_data, dtype=bool)
        connection_mask[x, y, z] = True
        
        # Generate mesh using marching cubes
        verts, faces, _, _ = marching_cubes(connection_mask, level=0.5)
        return verts, faces
    return None, None

# Generate mesh for the connected region
verts_conn, faces_conn = generate_connection_mesh(x, y, z)



Created a dash app with two main part(3D chart and 2D projection chart). And added a rotation angle adjustment slider to adjust the rotation angle of the 3D view

In [5]:
# Initialize Dash application
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    # 3D Visualization
    html.Div([
        html.H3("3D and 2D Projection of Cerebral Vessels and Aneurysms"),
        dcc.Graph(
            id='3d-plot',
            figure=go.Figure(),
            config={'scrollZoom': True, 'displayModeBar': True},
            style={'height': '40vh'}
        )
    ]),

    # 2D Projection
    html.Div([
        html.H3("2D Projection View"),
        dcc.Graph(
            id='2d-plot',
            figure=go.Figure(),
            config={'displayModeBar': False},
            style={'height': '40vh'}
        )
    ]),

    # Angle Adjustment
    html.Div([
        html.Label("Adjust Rotation Angle (0-360°):"),
        dcc.Slider(
            id='angle-slider',
            min=0,
            max=360,
            step=1,
            value=0,
            marks={i: f"{i}°" for i in range(0, 361, 40)},
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ]),
])



Calculate and update the view. Incuding three parts: camera view angle calculation, 2D projection and depth sorting.
Camera view angle calculation: Calculate the camera's eye position and upward direction based on the normal vector of the connected area and the rotation angle set by the user.
2D projection: Project the 3D point onto the 2D plane and calculate the projection position using the perspective projection rule.
Depth sorting: Sort all projected points according to their distance (depth) from the camera, so that they can be correctly superimposed in the 2D graph.

In [6]:
# Calculate camera view based on the normal vector and rotation angle
def calculate_camera_view(normal_vector, angle, zoom_factor=1.5):
    angle_rad = np.deg2rad(angle)

    # Calculate an orthogonal vector in the XY plane
    orthogonal_vector = np.array([-normal_vector[1], normal_vector[0], 0])
    orthogonal_vector /= np.linalg.norm(orthogonal_vector)

    # Rotate the orthogonal vector
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad), 0],
        [np.sin(angle_rad),  np.cos(angle_rad), 0],
        [0, 0, 1]
    ])
    rotated_orthogonal = np.dot(rotation_matrix, orthogonal_vector)

    # Determine the camera eye position and apply zoom
    eye = rotated_orthogonal
    eye /= np.linalg.norm(eye)
    eye *= zoom_factor

    return dict(x=eye[0], y=eye[1], z=eye[2]), dict(x=normal_vector[0], y=normal_vector[1], z=normal_vector[2])

# Project 3D points to 2D (using perspective projection)
def project_to_2d(x, y, z, camera_eye, camera_up):
    # Field of view scaling factor 
    fov = 1.0

    # Convert camera vectors to numpy arrays
    camera_eye = np.array([camera_eye['x'], camera_eye['y'], camera_eye['z']])
    camera_up = np.array([camera_up['x'], camera_up['y'], camera_up['z']])

    # Calculate the projection matrix related vector
    forward = camera_eye
    right = np.cross(camera_up, forward)
    up = np.cross(forward, right)

    right /= np.linalg.norm(right)
    up /= np.linalg.norm(up)

    # Projecting for points
    projected_x = np.dot(np.stack([x, y, z], axis=-1), right)
    projected_y = np.dot(np.stack([x, y, z], axis=-1), up)
    projected_x = np.nan_to_num(projected_x, nan=0)
    projected_y = np.nan_to_num(projected_y, nan=0)

    return projected_x, projected_y

# Calculate the depth of each point
def calculate_depth(x, y, z, camera_eye):
    camera_eye = np.array([camera_eye['x'], camera_eye['y'], camera_eye['z']])
    points = np.stack([x, y, z], axis=-1)
    depth = np.dot(points - camera_eye, camera_eye)
    return depth



Update 3D and 2D charts through Dash callback functions. The 3D view and 2D projection map will be dynamically updated when adjust the rotation angle.

In [7]:
# Callback function: Update 3D and 2D visualization
@app.callback(
    [Output('3d-plot', 'figure'), Output('2d-plot', 'figure')],
    [Input('angle-slider', 'value')]
)
def update_plots(angle):
    fig_3d = go.Figure()
    fig_2d = go.Figure()

    # Add the meshes for the cerebral vessels (Label 1) and aneurysms (Label 2)
    if verts1 is not None and faces1 is not None:
        fig_3d.add_trace(go.Mesh3d(
            x=verts1[:, 0], y=verts1[:, 1], z=verts1[:, 2],
            i=faces1[:, 0], j=faces1[:, 1], k=faces1[:, 2],
            color='red',
            opacity=1,
            name="Cerebral Vessels (Label 1)"
        ))

    if verts2 is not None and faces2 is not None:
        fig_3d.add_trace(go.Mesh3d(
            x=verts2[:, 0], y=verts2[:, 1], z=verts2[:, 2],
            i=faces2[:, 0], j=faces2[:, 1], k=faces2[:, 2],
            color='blue',
            opacity=1,
            name="Aneurysms (Label 2)"
        ))

    # Adding the mesh for the connect regions
    if verts_conn is not None and faces_conn is not None:
        fig_3d.add_trace(go.Mesh3d(
            x=verts_conn[:, 0], y=verts_conn[:, 1], z=verts_conn[:, 2],
            i=faces_conn[:, 0], j=faces_conn[:, 1], k=faces_conn[:, 2],
            color='green',
            opacity=1,
            name="Connection Region"
        ))

    # Add normal vector to line segment
    if len(x) > 0:
        start_point = np.array([center_x, center_y, center_z])
        end_point = start_point + normal_vector * 50  

        fig_3d.add_trace(go.Scatter3d(
            x=[start_point[0], end_point[0]],
            y=[start_point[1], end_point[1]],
            z=[start_point[2], end_point[2]],
            mode='lines',
            line=dict(color='purple', width=3),
            name="Normal Vector"
        ))

    # Get the camera's viewing angle
    camera_eye, camera_up = calculate_camera_view(normal_vector, angle, zoom_factor=2)

    # Project the points of cerebral vessels and aneurysms into 2D and calculate the depth
    projected_x1, projected_y1 = project_to_2d(x1, y1, z1, camera_eye, camera_up)
    projected_x2, projected_y2 = project_to_2d(x2, y2, z2, camera_eye, camera_up)
    depth1 = calculate_depth(x1, y1, z1, camera_eye)
    depth2 = calculate_depth(x2, y2, z2, camera_eye)

    # Projecting the connected area
    if verts_conn is not None:
        projected_x_conn, projected_y_conn = project_to_2d(verts_conn[:, 0], verts_conn[:, 1], verts_conn[:, 2], camera_eye, camera_up)
        depth_conn = calculate_depth(verts_conn[:, 0], verts_conn[:, 1], verts_conn[:, 2], camera_eye)
    else:
        projected_x_conn, projected_y_conn, depth_conn = np.array([]), np.array([]), np.array([])

    # Merge all points and sort them by depth (farthest to nearest)
    all_points = np.concatenate([
        np.column_stack((projected_x1, projected_y1, depth1, np.zeros_like(depth1))),  # Label 0: Cerebral vessels
        np.column_stack((projected_x2, projected_y2, depth2, np.ones_like(depth2))),   # Label 1: Aneurysms
        np.column_stack((projected_x_conn, projected_y_conn, depth_conn, 2 * np.ones_like(depth_conn)))  # Label 2: Connection region
    ])
    sorted_points = all_points[np.argsort(all_points[:, 2])]  # Depth descending sort

    # Map the corresponding color for each point
    color_map = {0: 'red', 1: 'blue', 2: 'green'}
    colors = np.array([color_map[label] for label in sorted_points[:, 3]])

    # Plot all points
    fig_2d.add_trace(go.Scatter(
        x=sorted_points[:, 0],
        y=sorted_points[:, 1],
        mode='markers',
        marker=dict(size=2, color=colors),
        name='2D Projection'
    ))

    # Update 3D layout
    fig_3d.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
            camera=dict(eye=camera_eye, up=camera_up)
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    # Update 2D layout
    fig_2d.update_layout(
        xaxis=dict(scaleanchor="y"),
        yaxis=dict(scaleanchor="x"),
        margin=dict(l=20, r=20, b=20, t=20)
    )

    return fig_3d, fig_2d

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)
